<a href="https://colab.research.google.com/github/zizilnam/Mini_Project_Web_Crawling_Naver_Stock/blob/main/%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 금 시세와 달러 환율 데이터 크롤링
- 둘 사이의 종가 및 등락율을 분석해서 둘 사이의 상관관계를 알아보는 EDA를 가정합니다.

In [ ]:
from urllib.request import urlopen
import requests
import bs4
import pandas as pd

## 국제 금 시세 데이터 크롤링

In [ ]:
# https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_GC&fdtc=2&page=1

index_cd = "CMDT_GC"
page_n = 1

naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd={index_cd}&fdtc=2&page={page_n}"

naver_index

'https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd=CMDT_GC&fdtc=2&page=1'

- url을 불러와서 데이터 열어보기

In [ ]:
src = urlopen(naver_index).read()

- BeautifulSoup의 힘을 빌려서 parser로 parsing 하기

In [ ]:
source = bs4.BeautifulSoup(src, 'lxml')
source

<html lang="ko">
<head>
<title>네이버 금융</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/finance.css" rel="stylesheet" type="text/css"/>
<script language="javascript">document.domain="naver.com";</script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/js/lcslog.js" type="text/javascript"></script>
</head>
<body>
<div class="section_exchange2">
<h3 class="h_today"><span>일별 시세</span></h3>
<table border="1" class="tbl_exchange today" summary="일별 시세 리스트">
<caption>일별 시세</caption>
<colgroup>
<col width="85"/>
<col width="83"/>
<col width="83"/>
<col width="82"/>
</colgroup>
<thead>
<tr>
<th class="th_today1

- prettify 함수를 사용하면 tab 기준으로 "이쁘게" 보인다는데 아직 아는 게 없어서 차이점을 못 느끼겠습니다.

In [ ]:
print(source.prettify())

<html lang="ko">
 <head>
  <title>
   네이버 금융
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="text/javascript" http-equiv="Content-Script-Type"/>
  <meta content="text/css" http-equiv="Content-Style-Type"/>
  <link href="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/css/finance.css" rel="stylesheet" type="text/css"/>
  <script language="javascript">
   document.domain="naver.com";
  </script>
  <script src="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript">
  </script>
  <script src="https://ssl.pstatic.net/imgstock/static.pc/20210721200146/js/lcslog.js" type="text/javascript">
  </script>
 </head>
 <body>
  <div class="section_exchange2">
   <h3 class="h_today">
    <span>
     일별 시세
    </span>
   </h3>
   <table border="1" class="tbl_exchange today" summary="일별 시세 리스트">
    <caption>
     일별 시세
    </caption>
    <colgroup>
     <col width="85"/>
     <co

- 수업 시간에 봤던 예제처럼 <td> 태그 안에 데이터가 있기 때문에 td 태그에 있는 모든 데이터를 가져옵니다.
- len으로 리스트 원소의 개수를 알아본다.

In [ ]:
td = source.find_all('td')
len(td)

28

### 날짜를 추출합니다
- 날짜 정보들은 td 태그 중 class가 "date" 인 태그에 있습니다.
- xpath로 접근해보고 추출해보기
- /html/body/div/table/tbody/tr[1]/td[1] = 2021.07.30

In [ ]:
# /html/body/div/table/tbody/tr[1]/td[1]
source.find_all('tr')[1].find_all('td')[0]

<td class="date">
		
		2021.08.02				
		</td>

- .text로 원하는 데이터를 구했지만 \n, \t가 섞여 있어서 .replace()로 제거합니다

In [ ]:
source.find_all('td', class_="date")[1].text

'\n\t\t\n\t\t2021.07.30\t\t\t\t\n\t\t'

In [ ]:
source.find_all('td', class_="date")[0].text.replace('\t','').replace('\n','')

'2021.08.02'

- 가져온 데이터를 yyyy.mm.dd 형태로 변환합니다.

In [ ]:
import datetime as dt

In [ ]:
date = source.find_all('td', class_="date")[0].text.replace('\t','').replace('\n','')

yyyy, mm, dd = [int(x) for x in date.split('.')]
yyyy, mm, dd

(2021, 8, 2)

In [ ]:
this_date = dt.date(yyyy, mm, dd)
this_date

datetime.date(2021, 8, 2)

In [ ]:
def date_format(date):
    yyyy, mm, dd = [int(x) for x in date.split('.')]
    return dt.date(yyyy, mm, dd)

### 종가를 추출합니다.

In [ ]:
source.find_all('td', class_='num')[0].text


'\n\t\t\t\n\t\t\t\t\n\t\t\t\t1,818.10\n\t\t\t\n\t\t'

- 이 데이터도 \n, \t, ' ' 가 섞여 있어서 replace(), strip()로 처리해줍니다.

In [ ]:
p = source.find_all('td', class_='num')[0].text.replace('\n','').replace('\t','').strip()
p

'1,818.10'

- source로 가져온 데이터를 보니  이 데이터도 num 클래스에 종가 말고 다른 데이터가 많습니다.

In [ ]:
source.find_all('td', class_='num')[1].text.replace('\n','').replace('\t','').strip()

'5.50'

In [ ]:
source.find_all('td', class_='num')[2].text.replace('\n','').replace('\t','').strip()

'+0.30%'

In [ ]:
source.find_all('td', class_='num')[3].text.replace('\n','').replace('\t','').strip()

'1,812.60'

In [ ]:
source.find_all('td', class_='num')[6].text.replace('\n','').replace('\t','').strip()

'1,831.20'

- 하나씩 확인해 보니 0, 3, 6, ... 이런 순서로 종가가 나옵니다.

### 등락률 불러오기

In [ ]:
source.find_all('td', class_='num')[11].text.replace('\n','').replace('\t','').replace('%','').strip()


'0.00'

- 2, 5, 8, 11, ... 순서로 등락률이 나타난다.

In [ ]:
prices[i*3-1].text

NameError: name 'prices' is not defined

- 이제 이런 규칙을 사용해 날짜와 종가와 등락률을 함께 불러옵니다.

In [ ]:
dates = source.find_all('td', class_='date')
len(dates)


7

In [ ]:
prices = source.find_all('td', class_='num')
len(prices)

21

In [ ]:
for i in range(len(dates)):
    this_date = dates[i].text
    this_date = date_format(this_date)   
    this_close = prices[i*3].text.replace('\n','').replace('\t','').replace(',','')
    this_close = float(this_close)
    this_ratio = prices[i*3+2].text.replace('\n','').replace('\t','').replace('%','')
    this_ratio = float(this_ratio)
    print(this_date, this_close, this_ratio)

2021-08-02 1818.1 0.3
2021-07-30 1812.6 -1.01
2021-07-29 1831.2 1.75
2021-07-28 1799.7 0.0
2021-07-27 1799.8 0.03
2021-07-26 1799.2 -0.14
2021-07-23 1801.8 -0.19


### 100 페이지에 있는 데이터 크롤링

In [ ]:
index_cd = "CMDT_GC"
page_n = 1  # page number
naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd={index_cd}&fdtc=2&page={page_n}"
src = urlopen(naver_index).read()
source = bs4.BeautifulSoup(src, 'lxml')
td = source.find_all('td')
dates = source.find_all('td', class_='date')
prices = source.find_all('td', class_='num')

for i in range(len(dates)):
    this_date = dates[i].text.replace('\m', '').replace('\t', '').strip()
    this_date =date_format(this_date)
    this_close = prices[i*3].text.replace('\n','').replace('\t','').replace(',','')
    this_close = float(this_close)
    this_ratio = prices[i*3+2].text.replace('\n','').replace('\t','').replace('%','')
    this_ratio = float(this_ratio)
    print(this_date, this_close, this_ratio)

2021-08-02 1818.1 0.3
2021-07-30 1812.6 -1.01
2021-07-29 1831.2 1.75
2021-07-28 1799.7 0.0
2021-07-27 1799.8 0.03
2021-07-26 1799.2 -0.14
2021-07-23 1801.8 -0.19


### 위에서 한 작업들 하나로 묶기

In [ ]:
def crawl_gold_index(index_cd, end_page):
    date_list = []
    price_list = []
    ratio_list = []
    for page_n in range(1, end_page+1):
        naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?marketindexCd={index_cd}&fdtc=2&page={page_n}"
        src = urlopen(naver_index).read()
        source = bs4.BeautifulSoup(src, 'lxml')
        td = source.find_all('td')
        dates = source.find_all('td', class_='date')
        prices = source.find_all('td', class_='num')
        for i in range(len(dates)):
            this_date = dates[i].text.replace('\m', '').replace('\t', '').strip()
            this_date =date_format(this_date)
            this_close = prices[i*3].text.replace('\n','').replace('\t','').replace(',','')
            this_close = float(this_close)
            this_ratio = prices[i*3+2].text.replace('\n','').replace('\t','').replace('%','')
            this_ratio = float(this_ratio)
            date_list.append(this_date)
            price_list.append(this_close)
            ratio_list.append(this_ratio)
        
    df = pd.DataFrame({'날짜' : date_list, "체결가" : price_list, "등락률" : ratio_list})
    return df



In [ ]:
crawl_gold_index("CMDT_GC", 20)

,날짜,체결가,등락률
0,2021-08-02,1818.1,0.30
1,2021-07-30,1812.6,-1.01
2,2021-07-29,1831.2,1.75
3,2021-07-28,1799.7,0.00
4,2021-07-27,1799.8,0.03
...,...,...,...
135,2021-01-19,1839.5,0.55
136,2021-01-15,1829.3,-1.13
137,2021-01-14,1850.3,-0.17
138,2021-01-13,1853.6,0.58


WTI도 비슷한 구조로 되어 있어서 한번 실험으로 불러와보겠습니다.

In [ ]:
crawl_gold_index("OIL_CL", 20)

,날짜,체결가,등락률
0,2021-08-02,71.26,-3.63
1,2021-07-30,73.95,0.44
2,2021-07-29,73.62,1.69
3,2021-07-28,72.39,1.03
4,2021-07-27,71.65,-0.36
...,...,...,...
135,2021-01-19,52.98,1.06
136,2021-01-15,52.36,-2.25
137,2021-01-14,53.57,1.24
138,2021-01-13,52.91,-0.56


원자재는 아무래도 다 비슷한 구조로 되어있는 것 같습니다.

구리도 한번?

In [ ]:
crawl_gold_index("CMDT_CDY", 20)

,날짜,체결가,등락률
0,2021-08-02,9737.0,-0.11
1,2021-07-30,9747.5,-0.34
2,2021-07-29,9781.0,0.87
3,2021-07-28,9697.0,-0.12
4,2021-07-27,9709.0,1.35
...,...,...,...
135,2021-01-18,7972.5,-0.09
136,2021-01-15,7979.5,-0.29
137,2021-01-14,8002.5,0.53
138,2021-01-13,7960.5,-0.28


## 달러 인덱스 (주요국 달러 환율)
- 달러 인덱스 = 50.14348112 * EURUSD^-0.576 * USDJPY^-0.136 * GBPUSD^-0.119 * USDCAD^-0.091 * USDSEK^-0.042 * USDCHF^-0.036
- 원자재의 가격과 가격 변동률을 크롤링 하는 방법 차용

- 유로달러 환율 구하기

https://finance.naver.com/marketindex/worldDailyQuote.nhn?fdtc=4&marketindexCd=FX_EURUSD&page=1

In [ ]:
index_cd = "FX_EURUSD"
page_n = 1
naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?fdtc=4&marketindexCd={index_cd}&page={page_n}"

In [ ]:
src = urlopen(naver_index).read()

In [ ]:
source = bs4.BeautifulSoup(src, 'lxml')
td= source.find_all('td')

In [ ]:
source.find_all('td', class_="date")

[<td class="date">
 		
 		2021.08.02				
 		</td>,
 <td class="date">
 		
 		2021.07.31				
 		</td>,
 <td class="date">
 		
 		2021.07.30				
 		</td>,
 <td class="date">
 		
 		2021.07.29				
 		</td>,
 <td class="date">
 		
 		2021.07.28				
 		</td>,
 <td class="date">
 		
 		2021.07.27				
 		</td>,
 <td class="date">
 		
 		2021.07.26				
 		</td>]

### 날짜 크롤링

In [ ]:
source.find_all('td', class_="date")[0].text

'\n\t\t\n\t\t2021.08.02\t\t\t\t\n\t\t'

In [ ]:
p = source.find_all('td', class_='date')[0].text.replace('\n','').replace('\t','').strip()
p

'2021.08.02'

In [ ]:
type(p)

str

### 종가 크롤링

In [ ]:
source.find_all('td', class_="num")[0].text

'\n\t\t\t\n\t\t\t\t1.1880\n\t\t\t\t\n\t\t\t\n\t\t'

In [ ]:
source.find_all('td', class_="num")[1].text

'\n 0.0011\n\t\t\t\t\n\t\t\t\n\t\t'

In [ ]:
source.find_all('td', class_="num")[2].text

'\n\t\t +0.09%\n\t\t'

In [ ]:
source.find_all('td', class_="num")[3].text

'\n\t\t\t\n\t\t\t\t1.1869\n\t\t\t\t\n\t\t\t\n\t\t'

0, 3, 6, 9, ... 순서로 종가가 크롤링 됩니다.

### 등락률 크롤링

In [ ]:
source.find_all('td', class_="num")[2]

<td class="num">
		 +0.09%
		</td>

In [ ]:
source.find_all('td', class_="num")[2].text

'\n\t\t +0.09%\n\t\t'

In [ ]:
dates = source.find_all('td', class_="date")
prices = source.find_all('td', class_="num")

## 해당 페이지 데이터 크롤링 방법

In [ ]:
for i in range(len(dates)):
    this_date = dates[i].text.replace('\n', '').replace('\t', '').strip()
    this_date = date_format(this_date)  
    this_close = prices[i*3].text.replace(',','')
    this_close = float(this_close)
    this_ratio = prices[i*3+2].text.replace('\n','').replace('\t','').replace('%','')
    this_ratio = float(this_ratio)
    print(this_date, this_close, this_ratio)

2021-08-02 1.188 0.09
2021-07-31 1.1869 0.1
2021-07-30 1.1857 -0.21
2021-07-29 1.1883 0.72
2021-07-28 1.1798 -0.23
2021-07-27 1.1826 0.15
2021-07-26 1.1808 0.31


### 100페이지에 있는 데이터 크롤링 하기

In [ ]:
index_cd = "FX_EURUSD"
page_n = 100
naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?fdtc=4&marketindexCd={index_cd}&page={page_n}"
src = urlopen(naver_index).read()
source = bs4.BeautifulSoup(src, 'lxml')
dates = source.find_all('td', class_="date")
prices = source.find_all('td', class_="num")
for i in range(len(dates)):
    this_date = dates[i].text
    this_date = date_format(this_date)   
    this_close = prices[i*3].text.replace(',','')
    this_close = float(this_close)
    print(this_date, this_close)

2019-05-16 1.1176
2019-05-15 1.1209
2019-05-14 1.1211
2019-05-13 1.1242
2019-05-11 1.1235
2019-05-10 1.1245
2019-05-09 1.123


In [ ]:
def crawl_dollar_index(index_cd, end_page):
    date_list = []
    price_list = []
    ratio_list = []
    for page_n in range(1, end_page+1):
        naver_index = f"https://finance.naver.com/marketindex/worldDailyQuote.nhn?fdtc=4&marketindexCd={index_cd}&page={page_n}"
        src = urlopen(naver_index).read()
        source = bs4.BeautifulSoup(src, 'lxml')
        td = source.find_all('td')
        dates = source.find_all('td', class_='date')
        prices = source.find_all('td', class_='num')
        for i in range(len(dates)):
            this_date = dates[i].text.replace('\n', '').replace('\t', '').strip()
            this_date =date_format(this_date)
            this_close = prices[i*3].text.replace('\n','').replace('\t','').replace(',','')
            this_close = float(this_close)
            this_ratio = prices[i*3+2].text.replace('\n','').replace('\t','').replace('%','')
            this_ratio = float(this_ratio)
            date_list.append(this_date)
            price_list.append(this_close)
            ratio_list.append(this_ratio)
    df = pd.DataFrame({'날짜' : date_list, "체결가" : price_list, "등락률" : ratio_list})
    return df
crawl_dollar_index('FX_EURUSD', 100)

,날짜,체결가,등락률
0,2021-08-02,1.1880,0.09
1,2021-07-31,1.1869,0.10
2,2021-07-30,1.1857,-0.21
3,2021-07-29,1.1883,0.72
4,2021-07-28,1.1798,-0.23
...,...,...,...
695,2019-05-14,1.1211,-0.27
696,2019-05-13,1.1242,0.06
697,2019-05-11,1.1235,-0.08
698,2019-05-10,1.1245,0.13


### EURUSD 달러 환율 데이터

In [ ]:
crawl_dollar_index('FX_EURUSD', 100)

,날짜,체결가,등락률
0,2021-08-02,1.1880,0.09
1,2021-07-31,1.1869,0.10
2,2021-07-30,1.1857,-0.21
3,2021-07-29,1.1883,0.72
4,2021-07-28,1.1798,-0.23
...,...,...,...
695,2019-05-14,1.1211,-0.27
696,2019-05-13,1.1242,0.06
697,2019-05-11,1.1235,-0.08
698,2019-05-10,1.1245,0.13


### JPYUSD 환율 데이터

In [ ]:
crawl_dollar_index('FX_USDJPY', 100)

,날짜,체결가,등락률
0,2021-08-02,109.25,-0.39
1,2021-07-31,109.68,-0.06
2,2021-07-30,109.75,0.12
3,2021-07-29,109.61,-0.43
4,2021-07-28,110.09,0.24
...,...,...,...
695,2019-05-14,109.57,0.41
696,2019-05-13,109.12,-0.74
697,2019-05-11,109.94,0.31
698,2019-05-10,109.59,0.03


### GBPUSD 환율 데이터

In [ ]:
crawl_dollar_index('FX_GBPUSD', 100)

,날짜,체결가,등락률
0,2021-08-02,1.3896,-0.05
1,2021-07-31,1.3903,-0.01
2,2021-07-30,1.3905,-0.49
3,2021-07-29,1.3974,0.74
4,2021-07-28,1.3871,-0.08
...,...,...,...
695,2019-05-14,1.2918,-0.53
696,2019-05-13,1.2987,-0.06
697,2019-05-11,1.2996,-0.26
698,2019-05-10,1.3031,0.09


### USDCAD 환율 데이터

In [ ]:
crawl_dollar_index('FX_USDCAD', 100)

,날짜,체결가,등락률
0,2021-08-02,1.2489,0.09
1,2021-07-31,1.2477,-0.04
2,2021-07-30,1.2482,0.29
3,2021-07-29,1.2445,-1.06
4,2021-07-28,1.2579,0.11
...,...,...,...
695,2019-05-14,1.3467,0.17
696,2019-05-13,1.3443,0.16
697,2019-05-11,1.3421,-0.08
698,2019-05-10,1.3433,-0.39


### USDSEK 환율 데이터

In [ ]:
crawl_dollar_index('FX_USDSEK', 100)

### USDCHF 환율 데이터

In [ ]:
crawl_dollar_index('FX_USDCHF', 100)

## 국제 금이 내가 가진 주식의 헷징 수단이 될 수 있는가

- 내가 가진 주식들 크롤링하기( 네이버, 펄어비스)

In [ ]:
url_stock = "https://finance.naver.com/item/sise_day.nhn?code=035420&page=1"
stock_src = urlopen(url_stock).read()
bs4.BeautifulSoup(stock_src, 'lxml')

In [ ]:
stock_cd = '035420'
page_n = 1

url_stock = f"https://finance.naver.com/item/sise_day.nhn?code={stock_cd}&page={page_n}"
headers = {
    'authority': 'finance.naver.com',
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}
res = requests.get(url_stock, headers=headers)
source = res.text
src = bs4.BeautifulSoup(source, 'lxml')

In [ ]:
src = bs4.BeautifulSoup(source, 'lxml')
src

### 날짜 크롤링

In [ ]:
src.find_all('table')[0].find_all('tr')[2].find_all('td')[0]

## /html/body/table[1]/tbody/tr[3]/td[1]/span

In [ ]:
src.find_all('td', align='center')[0]

In [ ]:
a = src.find_all('span', class_='tah p10 gray03')[0].text

In [ ]:
date_format(a)

### 종가 크롤링

In [ ]:
src.find_all('td', class_='num')[0]

## /html/body/table[1]/tbody/tr[3]/td[2]/span

In [ ]:
src.find_all('td', class_='num')[6]

In [ ]:
src.find_all('td', class_='num')[12].text


0, 6, 12, ... 번째 원소들이 종가

### 해당 페이지 크롤링

In [ ]:
dates = src.find_all('td', align='center')
prices = src.find_all('td', class_='num')
len(prices)
prices[52]

In [ ]:
for i in range(len(dates)):
    this_time = dates[i].text
    this_time = date_format(this_time)
    this_close = prices[i*6].text.replace(',','')
    this_close = float(this_close)
    print(this_time, this_close)

### 100 페이지에 있는 데이터 크롤링

In [ ]:
stock_cd = "035420"
page_n = 100
url_stock = f"https://finance.naver.com/item/sise_day.nhn?code={stock_cd}&page={page_n}"
headers = {
    'authority': 'finance.naver.com',
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}
res = requests.get(url_stock, headers=headers)
source = res.text
src = bs4.BeautifulSoup(source, 'lxml')
dates = src.find_all('td', align='center')
prices = src.find_all('td', class_='num')
for i in range(len(dates)):
    this_time = dates[i].text
    this_time = date_format(this_time)
    this_price = prices[i*6].text.replace(',','')
    this_price = float(this_price)
    print(this_time, this_price)

In [ ]:
def crawl_stock_price(stock_cd, end_page):
    time_list = []
    price_list = []
    for page_n in range(1, end_page+1):
        
        url_stock = f"https://finance.naver.com/item/sise_day.nhn?code={stock_cd}&page={page_n}"
        headers = {
        'authority': 'finance.naver.com',
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
        }
        res = requests.get(url_stock, headers=headers)
        source = res.text
        src = bs4.BeautifulSoup(source, 'lxml')
        dates = src.find_all('td', align='center')
        prices = src.find_all('td', class_='num')
        for i in range(len(dates)):
            this_time = dates[i].text
            this_time = date_format(this_time)
            this_price = prices[i*6].text.replace(',','')
            this_price = float(this_price)
            time_list.append(this_time)
            price_list.append(this_price)
    df = pd.DataFrame({"날짜" : time_list, "종가" : price_list})
    return df


In [ ]:
stock_cd = "263750"
page_n = 50
url_stock = f"https://finance.naver.com/item/sise_day.nhn?code={stock_cd}&page={page_n}"
headers = {
'authority': 'finance.naver.com',
'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}
res = requests.get(url_stock, headers=headers)
source = res.text
src = bs4.BeautifulSoup(source, 'lxml')
dates = src.find_all('td', align='center')
prices = src.find_all('td', class_='num')
a = src.find_all('td', align = 'center')[1].text
type(a)
this_time = src.find_all('td', align = 'center')[0].text
date_format(this_time)
for i in range(len(dates)):
    this_time = dates[i].text
    this_time = date_format(this_time)
    this_close = prices[i*6].text.replace(',','')
    this_clsoe = float(this_close)
    print(this_time, this_close)

### 펄어비스 주가

In [ ]:
crawl_stock_price("263750",95)

###  네이버 주가

In [ ]:
crawl_stock_price('035420', 100)

## 네이버 증권에서 뉴스 크롤링

In [ ]:
stock_cd = '263750'
page_n = '1'
news_url = f"https://finance.naver.com/item/news_news.nhn?code={stock_cd}&page={page_n}&sm=title_entity_id.basic&clusterId="
source = urlopen(news_url).read()

In [ ]:
src = bs4.BeautifulSoup(source, 'lxml')

In [ ]:
td = src.find_all('td')
td

[<td class="title">
 <a class="tit" href="/item/news_read.nhn?article_id=0000750870&amp;office_id=366&amp;code=263750&amp;page=1&amp;sm=title_entity_id.basic" onclick="clickcr(this,'stn.ntitclust','','',event);" target="_top">펄어비스, 신작 ‘붉은사막’ 출시 연기 소식에 4% 넘게 하락</a>
 </td>,
 <td class="info">조선비즈</td>,
 <td class="date"> 2021.07.30 14:34</td>,
 <td colspan="3">
 <table class="type5">
 <caption>연관기사 목록</caption>
 <colgroup>
 <col/>
 <col width="130px"/>
 <col width="110px"/>
 </colgroup>
 <tbody>
 <tr>
 <td class="title">
 <a class="tit" href="/item/news_read.nhn?article_id=0004624278&amp;office_id=008&amp;code=263750&amp;page=1&amp;sm=title_entity_id.basic" onclick="clickcr(this,'stn.ntitclustsub','','',event);" target="_top"><span class="ico_reply"></span>펄어비스, 기대작 '붉은사막' 출시 연기에 5.48%↓</a>
 </td>
 <td class="info">머니투데이</td>
 <td class="date"> 2021.07.30 10:12</td>
 </tr>
 <tr class="hide_news none">
 <td class="title">
 <a class="tit" href="/item/news_read.nhn?article_id=0004998558&amp

## 증권 뉴스 날짜 크롤링

In [ ]:
src.find_all('td', class_='date')[0].text.strip().split()[0]

'2021.07.30'

In [ ]:
src.find_all('td', class_='date')[4].text.strip().split()[0]

'2021.07.29'

## 증권 뉴스 제목 크롤링

In [ ]:
src.find_all('td', class_='title')[0].text.replace('\n','')

'펄어비스, 신작 ‘붉은사막’ 출시 연기 소식에 4% 넘게 하락'

## 증권 뉴스 정보제공 신문사 크롤링

In [ ]:
src.find_all('td', class_='info')[0].text

'머니투데이'

## 증권 뉴스 내용 크롤링 하기(가장 많이 쓰인 단어 베스트 5)

- 링크 얻기

In [ ]:
art = src.find_all('td', class_='title')[0].find_all('a', class_='tit')[0]
type(art)
art

<a class="tit" href="/item/news_read.nhn?article_id=0004533907&amp;office_id=008&amp;code=263750&amp;page=20&amp;sm=title_entity_id.basic" onclick="clickcr(this,'stn.ntitclust','','',event);" target="_top">펄어비스, 붉은사막 기대감에 기업가치 상승…목표가↑-SK</a>

In [ ]:
art["href"]

'/item/news_read.nhn?article_id=0004533907&office_id=008&code=263750&page=20&sm=title_entity_id.basic'

In [ ]:
art = src.find_all('td', class_='title')[1].find_all('a', class_='tit')[0]
type(art)
art["href"]

'/item/news_read.nhn?article_id=0003860806&office_id=011&code=263750&page=20&sm=title_entity_id.basic'

In [ ]:
src.find_all('td', class_='title')[1].find_all('a', class_='tit')[0]["href"]

'/item/news_read.nhn?article_id=0003860806&office_id=011&code=263750&page=20&sm=title_entity_id.basic'

- 링크 안의 내용 가져오기

In [ ]:
news_link = "https://finance.naver.com" + src.find_all('td', class_='title')[0].find_all('a', class_='tit')[0]["href"]

news_link
# https://finance.naver.com/item/news_read.nhn?article_id=0000750870&office_id=366&code=263750&page=1&sm=title_entity_id.basic

'https://finance.naver.com/item/news_read.nhn?article_id=0004533907&office_id=008&code=263750&page=20&sm=title_entity_id.basic'

In [ ]:
res = requests.get(news_link)
art_source = res.text
art_src = bs4.BeautifulSoup(art_source, 'lxml')
art = art_src.find('div', id='news_read')
type(art.text)

str

In [ ]:
news_data = art.text.split()
def most_frequent(news_data):
    return max(news_data, key=news_data.count)
most_frequent(news_data)

'검은사막'

## 해당 페이지 데이터 크롤링

In [ ]:
dates = src.find_all('td', class_='date')

In [ ]:
titles = src.find_all('td', class_='title')

In [ ]:
infos = src.find_all('td', class_='info')

In [ ]:
for i in range(len(dates)):
    news_date = dates[i].text.strip().split()[0]
    news_date = date_format(news_date)
    news_title = titles[i].text.replace('\n','')
    news_info = info[i].text
    news_link = "https://finance.naver.com" + src.find_all('td', class_='title')[0].find_all('a', class_='tit')[0]["href"]
    news_source = urlopen(news_link).read()
    news_src = bs4.BeautifulSoup(news_source)
    news_art = news_src.find('div', id='news_read').text.split()
    word_dict = dict()
    for word in news_art:
        word_dict[word] = word_dict.get(word, 0) + 1
    a = sorted(word_dict, key= lambda x : word_dict[x], reverse=True)
    news_word1 = a[0]
    news_word2 = a[1]
    news_word3 = a[2]
    news_word4 = a[3]
    news_word5 = a[4]
#     news_word1 = most_frequent(news_art)
#     news_art.remove(news_word1)
#     news_word2 = most_frequent(news_art)
#     news_art.remove(news_word2)
#     news_word3 = most_frequent(news_art)
    print(news_date, news_title, news_info, news_word1, news_word2, news_word3, news_word4, news_word5)

2021-02-17 "펄어비스, 붉은사막 흥행 기대···목표가↑" 조선비즈 및 올해 연구원은 기존 KTB증권은
2021-02-17 펄어비스, 신작 '붉은사막' 예상 판매량↑…목표가도↑-KTB 머니투데이 및 올해 연구원은 기존 KTB증권은
2021-02-17 펄어비스, 신작 '붉은사막' 흥행 기대…조정시 매수 -신한 이데일리 및 올해 연구원은 기존 KTB증권은
2021-02-17 [클릭 e종목]"펄어비스, 붉은사막 초기 판매량 상향조정…목표가↑" 조선비즈 및 올해 연구원은 기존 KTB증권은
2021-02-16 펄어비스, 5대1 액면분할 결정 이데일리 및 올해 연구원은 기존 KTB증권은
2021-02-16 펄어비스, 5대 1 액면분할 결정 매일경제 및 올해 연구원은 기존 KTB증권은
2021-02-16 펄어비스 작년 영업익 4.4% 증가 이데일리 및 올해 연구원은 기존 KTB증권은
2021-02-16 펄어비스, 올해 '붉은사막'만 보고 간다…"트리플A급 성적 자신" 헤럴드경제 및 올해 연구원은 기존 KTB증권은
2021-02-16 [종합] 펄어비스, 올해 ‘붉은사막’ 집중…도약 예열 ‘이상무’ 이데일리 및 올해 연구원은 기존 KTB증권은
2021-02-16 [펄어비스 컨콜] “붉은사막, 트리플A급 성과 달성 ‘목표’” 파이낸셜뉴스 및 올해 연구원은 기존 KTB증권은
2021-02-16 [컨콜] 펄어비스, ‘붉은사막’ 기대감…‘도깨비·플랜8’ 해마다 1종씩... 아시아경제 및 올해 연구원은 기존 KTB증권은
2021-02-16 펄어비스, 작년 영업익 1573억…해외매출 비중 77% 이데일리 및 올해 연구원은 기존 KTB증권은
2021-02-16 ‘검은사막’ 펄어비스 4분기 영업익 200억…전년比 30% 급락 이데일리 및 올해 연구원은 기존 KTB증권은
2021-02-16 펄어비스, 지난해 영업이익 1,573억 원... 전년比 4.4% 늘어 헤럴드경제 및 올해 연구원은 기존 KTB증권은
2021-02-16 펄어비스, 작년 매출 4888억원…전년비 매출↓·영업익↑ 머니

IndexError: list index out of range

## 100페이지에 있는 데이터 크롤링

In [ ]:
page_n = 20
stock_cd = '263750'
news_url = f"https://finance.naver.com/item/news_news.nhn?code={stock_cd}&page={page_n}&sm=title_entity_id.basic&clusterId="
source = urlopen(news_url).read()
src = bs4.BeautifulSoup(source, 'lxml')
dates = src.find_all('td', class_='date')
titles = src.find_all('td', class_='title')
infos = src.find_all('td', class_='info')
for i in range(len(dates)):
    news_date = dates[i].text.strip().split()[0]
    news_date = date_format(news_date)
    news_title = titles[i].text.replace('\n','')
    news_info = infos[i].text
    news_link = "https://finance.naver.com" + src.find_all('td', class_='title')[0].find_all('a', class_='tit')[0]["href"]
    news_source = urlopen(news_link).read()
    news_src = bs4.BeautifulSoup(news_source)
    news_art = news_src.find('div', id='news_read').text.split()
    word_dict = dict()
    for word in news_art:
        word_dict[word] = word_dict.get(word, 0) + 1
    a = sorted(word_dict, key= lambda x : word_dict[x], reverse=True)
    news_word1 = a[0]
    news_word2 = a[1]
    news_word3 = a[2]
    news_word4 = a[3]
    news_word5 = a[4]
#     news_word1 = most_frequent(news_art)
#     news_art.remove(news_word1)
#     news_word2 = most_frequent(news_art)
#     news_art.remove(news_word2)
#     news_word3 = most_frequent(news_art)
    print(news_date, news_title, news_info, news_word1, news_word2, news_word3, news_word4, news_word5)

2021-01-27 [특징주]엔씨소프트·넷마블·펄어비스 급등…계속되는 게임株 강세 헤럴드경제 올해 것으로 있다. 및 대한
2021-01-26 펄어비스, 붉은사막 기대감에 기업가치 상승…목표가↑-SK 머니투데이 올해 것으로 있다. 및 대한
2021-01-22 [특징주] 신작효과·실적 개선 전망에 웹젠·펄어비스 등 동반 상승 서울경제 올해 것으로 있다. 및 대한
2021-01-22 펄어비스, 신작 '붉은 사막' 모멘텀 등 긍정적…목표가↑ -이베스트 이데일리 올해 것으로 있다. 및 대한
2021-01-21 펄어비스의 ‘검은사막’ 북미·유럽 직접 서비스…이번에도 성공할까 매일경제 올해 것으로 있다. 및 대한
2021-01-14 펄어비스 ‘섀도우 아레나’, 팀 협업 플레이 강화 이데일리 올해 것으로 있다. 및 대한
2021-01-05 펄어비스·카카오게임즈·컴투스 게임중견 3인방, 대형 신작 승부수 매일경제 올해 것으로 있다. 및 대한
2021-01-04 펄어비스 ‘검은사막’, 북미·유럽 스팀 4위 ‘기염’ 이데일리 올해 것으로 있다. 및 대한
2021-01-04 펄어비스 ‘검은사막’ 북미·유럽 스팀 MMO 장르 판매 4위 매일경제 올해 것으로 있다. 및 대한
2020-12-30 펄어비스 ‘검은사막’, 업데이트 통했다…신규 이용자 125% 증가 매일경제 올해 것으로 있다. 및 대한
2020-12-23 펄어비스, 새해 ‘붉은사막’으로 승부한다 파이낸셜뉴스 올해 것으로 있다. 및 대한


## 데이터들 크롤링하기

In [ ]:
def crawl_news_data(stock_cd, end_page):
    date_list = []
    title_list = []
    info_list = []
    word_list1 = []
    word_list2 = []
    word_list3 = []
    word_list4 = []
    word_list5 = []
    for i in range(1, end_page+1):
        page_n = i
        news_url = f"https://finance.naver.com/item/news_news.nhn?code={stock_cd}&page={page_n}&sm=title_entity_id.basic&clusterId="
        source = urlopen(news_url).read()
        src = bs4.BeautifulSoup(source, 'lxml')
        dates = src.find_all('td', class_='date')
        titles = src.find_all('td', class_='title')
        infos = src.find_all('td', class_='info')
        for i in range(len(dates)):
            news_date = dates[i].text.strip().split()[0]
            news_date = date_format(news_date)
            news_title = titles[i].text.replace('\n','')
            news_info = infos[i].text
            news_link = "https://finance.naver.com" + src.find_all('td', class_='title')[0].find_all('a', class_='tit')[0]["href"]
            news_source = urlopen(news_link).read()
            news_src = bs4.BeautifulSoup(news_source)
            news_art = news_src.find('div', id='news_read').text.split()
            word_dict = dict()
            for word in news_art:
                word_dict[word] = word_dict.get(word, 0) + 1
            a = sorted(word_dict, key= lambda x : word_dict[x], reverse=True)
            news_word1 = a[0]
            news_word2 = a[1]
            news_word3 = a[2]
            news_word4 = a[3]
            news_word5 = a[4]
#             news_word1 = most_frequent(news_art)
#             news_art.remove(news_word1)
#             news_word2 = most_frequent(news_art)
#             news_art.remove(news_word2)
#             news_word3 = most_frequent(news_art)
            date_list.append(news_date)
            title_list.append(news_title)
            info_list.append(news_info)
            word_list1.append(news_word1)
            word_list2.append(news_word2)
            word_list3.append(news_word3)
            word_list4.append(news_word4)
            word_list5.append(news_word5)
    df = pd.DataFrame({"날짜" : date_list, "뉴스 제목" : title_list, "제공" : info_list, '키워드1' : word_list1, '키워드2' : word_list2, '키워드3' : word_list3,
                      "키워드4" : word_list4, "키워드5" : word_list5})
    return df
crawl_news_data(263750, 10)


,날짜,뉴스 제목,제공,키워드1,키워드2,키워드3,키워드4,키워드5
0,2021-08-03,게임株까지 덮친 ‘中공산당 리스크’ ···펄어비스·위메이드 와르르,서울경제,중국,게임,국내,발급,규제
1,2021-07-30,"펄어비스, 신작 ‘붉은사막’ 출시 연기 소식에 4% 넘게 하락",조선비즈,중국,게임,국내,발급,규제
2,2021-07-30,"펄어비스, 기대작 '붉은사막' 출시 연기에 5.48%↓",머니투데이,중국,게임,국내,발급,규제
3,2021-07-30,"[특징주]펄어비스, '붉은사막' 출시 연기에 4%대↓",이데일리,중국,게임,국내,발급,규제
4,2021-07-29,펄어비스 신작 ‘붉은사막’ 출시 연기,조선비즈,중국,게임,국내,발급,규제
...,...,...,...,...,...,...,...,...
177,2021-02-16,"[공시] 펄어비스, 4Q 영업익 199억원...전년比 29.9%↓",파이낸셜뉴스,및,올해,연구원은,기존,KTB증권은
178,2021-02-16,"펄어비스, 지난해 영업익 1573억…전년比 4.4%↑ ‘선방’",매일경제,및,올해,연구원은,기존,KTB증권은
179,2021-02-16,"펄어비스, 작년 영업이익 1573억...전년比 4.4% ↑",아시아경제,및,올해,연구원은,기존,KTB증권은
180,2021-02-11,"코스닥 게임 대장株 펄어비스, 실적 타고 더 오를까",이데일리,및,올해,연구원은,기존,KTB증권은


## 네이버 뉴스 크롤링하기
- 네이버 코드  035420

In [ ]:
crawl_news_data('035420', 50)

,날짜,뉴스 제목,제공,키워드1,키워드2,키워드3,키워드4,키워드5
0,2021-08-03,이제 네이버에 'OOO 코로나' 검색해도 연관검색어 안뜬다,한국경제,인물명이,포함된,앞으로는,'코로나,'OOO
1,2021-08-03,네이버에서 인물명 검색하면 연관검색어 안뜬다,머니투데이,인물명이,포함된,앞으로는,'코로나,'OOO
2,2021-08-03,"""맞수에게 배우자"" 닮아가는 네이버·쿠팡",서울경제,인물명이,포함된,앞으로는,'코로나,'OOO
3,2021-08-03,"비즈넵 운영사 에멘탈, 네이버 등서 60억원 투자 유치",서울경제,인물명이,포함된,앞으로는,'코로나,'OOO
4,2021-08-03,"[fn마켓워치]네이버·고릴라PE, 경영관리 자동화 서비스에 투자",파이낸셜뉴스,인물명이,포함된,앞으로는,'코로나,'OOO
...,...,...,...,...,...,...,...,...
821,2021-06-25,네이버·카카오 뉴스 편집권 폐지?…구글엔 '속수무책',머니투데이,네이버,최,COO는,조사,대한
822,2021-06-24,카카오 네이버가 다했네…코스피 이달 시총 증가 40% 차지,매일경제,네이버,최,COO는,조사,대한
823,2021-06-24,"네이버, 왓패드와 시너지 본격화…콘텐츠 1000억 투자",이데일리,네이버,최,COO는,조사,대한
824,2021-06-24,"네이버, 웹툰-왓패드스튜디오 통합..글로벌IP 승부",파이낸셜뉴스,네이버,최,COO는,조사,대한


## 삼성전자 뉴스 크롤링

In [ ]:
crawl_news_data('005930', 50)

,날짜,뉴스 제목,제공,키워드1,키워드2,키워드3,키워드4,키워드5
0,2021-08-03,“하라는 거야 말라는 거야?” 삼성 체험 서비스 갈수록 ‘깐깐’,헤럴드경제,수,서비스를,등,있는,체험
1,2021-08-03,"‘갤럭시 얼리버드 투고 서비스’ 운영…삼성, 폴더블폰 마케팅 ‘시동’",이데일리,수,서비스를,등,있는,체험
2,2021-08-03,"'김연경' 갤워치4 갤폴드3, '3일 공짜폰' 됐다…삼성, 제품체험 ...",매일경제,수,서비스를,등,있는,체험
3,2021-08-03,인도에 공들이는 삼성…KLE대학에 'AI 연구소' 세운 까닭,머니투데이,수,서비스를,등,있는,체험
4,2021-08-03,"삼성전자, 印대학과 첨단기술 싹 틔운다…AI·머신러닝 연구소 세워",아시아경제,수,서비스를,등,있는,체험
...,...,...,...,...,...,...,...,...
881,2021-07-22,"존림 삼바 사장 ""모더나 위탁생산 亞서 삼성만 가능""",이데일리,'비스포크,있다.,에어컨,삼성,위해
882,2021-07-22,"美 오스틴 당국 “삼성과 협상시작, 8월 중 합의”…경제계 “이재용 부...",헤럴드경제,'비스포크,있다.,에어컨,삼성,위해
883,2021-07-22,삼성전자 부진해도…삼성그룹株 펀드는 ‘꿋꿋’,아시아경제,'비스포크,있다.,에어컨,삼성,위해
884,2021-07-22,[특징주] 삼성전자 ‘8만 전자’ 회복할까…SK하이닉스도 강세,헤럴드경제,'비스포크,있다.,에어컨,삼성,위해


## SK하이닉스 뉴스 크롤링

In [ ]:
crawl_news_data('000660', 50)

,날짜,뉴스 제목,제공,키워드1,키워드2,키워드3,키워드4,키워드5
0,2021-08-03,삼성전자 두달만에 2%대 상승… SK하이닉스는 3%↑,파이낸셜뉴스,삼성전자,지난,것은,이후,주가는
1,2021-08-03,"삼성전자·포스코·SK하이닉스·롯데, 청년 고용 위해 정부와 맞손",한국경제,삼성전자,지난,것은,이후,주가는
2,2021-08-03,"SK하이닉스, ESG 캠페인으로 '디지털애드어워즈' 대상",아시아경제,삼성전자,지난,것은,이후,주가는
3,2021-08-03,"[특징주]삼성전자, 12거래일 만에 장중 ‘8만전자’ 회복…SK하이닉스...",헤럴드경제,삼성전자,지난,것은,이후,주가는
4,2021-08-03,"[특징주] 삼전, 13거래일 만에 8만원선 회복...SK하이닉스도 3%...",서울경제,삼성전자,지난,것은,이후,주가는
...,...,...,...,...,...,...,...,...
837,2021-04-15,'환경·노동 규제완화' 반도체 특별법 나온다...첫 수혜는 SK하이닉스,조선비즈,심사를,반도체,낸드플래시,최근,기자
838,2021-04-15,"최태원의 SK, '하이닉스 직접 지배' 왜 보류했나",머니투데이,심사를,반도체,낸드플래시,최근,기자
839,2021-04-15,"SKT 중간 지주사 전환에도 하이닉스 투자 규제 여젼…""국내투자 역차...",이데일리,심사를,반도체,낸드플래시,최근,기자
840,2021-04-15,"풀HD급 영화 1초에 저장...SK하이닉스, 기업용 SSD 신제품 양산",헤럴드경제,심사를,반도체,낸드플래시,최근,기자
